<a href="https://colab.research.google.com/github/xslittlemaggie/Deep-Learning-Machine-Learning-Projects/blob/master/Save_%26_load_pre_trained_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Step 0: Import libraries

In [2]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, Adam, Adadelta
from keras import backend as K
from keras.utils import np_utils
import numpy as np
import random
% matplotlib inline
import matplotlib.pyplot as plt

# Confusion matrix result
from sklearn.metrics import classification_report, confusion_matrix

# visualizating of confusion matrix
import seaborn as sn
import pandas as pd

# upload new images
import urllib.request
import os
import cv2

Using TensorFlow backend.


## Step 1: Load cifar10 data

In [3]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 4s 0us/step


In [4]:
print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)

print("x_test shape:", x_test.shape)
print("y_test shape:", y_test.shape)

x_train shape: (50000, 32, 32, 3)
y_train shape: (50000, 1)
x_test shape: (10000, 32, 32, 3)
y_test shape: (10000, 1)


## Step 2: Data preprocessing

The pixel values are in the range of 0 to 255 for each of the red, green and blue channels.

It is good practice to work with normalized data.

We can easily normalize to range 0 to 1 by dividing each value by the maximum observation which is 255.

In [0]:
if K.image_data_format() == "channels_first":
  x_train = x_train.reshape(x_train.shape[0], 3, 32, 32)
  x_test = x_test.reshape(x_test.shape[0], 3, 32, 32)
  input_shape = (3, 32, 32)
  
else:
  x_train = x_train.reshape(x_train.shape[0], 32, 32, 3)
  x_test = x_test.reshape(x_test.shape[0], 32, 32, 3)
  input_shape = (32, 32, 3)
  
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 225.
x_test /= 225.

# convert class vectors to binary class matrics
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

## Load and use weights from a checkpoint ('weights.best.hdf5' trained before)

In [7]:
num_classes = 10

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))

model.add(Conv2D(32, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25)) # Add dropout layer here

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv2D(64, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25)) # Add dropout layer here

model.add(Flatten())
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5)) # Add dropout layer here
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# load pre-trained weights 
model.load_weights('weights.best.hdf5')
model.compile(loss = 'categorical_crossentropy', # better loss function for neural networks
            optimizer = 'adam', # Adam optimizer with 1.0e-4 learning rate
            metrics = ['accuracy'])
print('Created model and loaded weights from file')  

# load pima indians dataset
scores = model.evaluate(x_test, y_test, verbose = 0)
print('Test Acc: {}'.format(scores[1]))

OSError: ignored